In [1]:
# Load Hyperparameters and global variables

In [6]:
import tensorflow as tf
import numpy as np
import argparse
import logging
import pprint # pretty print python objects
import sys
import os

FLAGS = {
    "config_file": "snli.config",
    "buffer_size": 10000,
    "batch_size": 20,
    "snli_link": "https://nlp.stanford.edu/projects/snli/snli_1.0.zip",
    "snli_zipfilename": "snli_1.0.zip",
    "snli_trainfilename": "snli_1.0_train.txt",
    "snli_validatefilename": "snli_1.0_dev.txt",
    "snli_testfilename": "snli_1.0_test.txt",
    "word_embeddings_link": "http://nlp.stanford.edu/data/glove.6B.zip",
    "word_embeddings_zipfilename": "glove.6B.zip",
    "word_embeddings_txtfilename": "glove.6B.50d.txt",
    "max_hypothesis_length": 30,
    "embedding_size": 300,
    "max_features": 50000,
    "max_premise_length": 30,
    "hidden_length": 64,
    "num_epochs": 5
}

def create_logger():
    log = logging.getLogger() # root logger
    log.setLevel(logging.DEBUG)
    formatter = logging.Formatter(fmt="%(asctime)s : %(levelname)s %(message)s")
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    log.addHandler(handler)
    return logging.getLogger()

pp = pprint.PrettyPrinter(indent=2)
logger = create_logger()
glove_wordmap = {}
glove_wordmap_size = 0

print(pp.pformat(FLAGS))

{ 'batch_size': 20,
  'buffer_size': 10000,
  'config_file': 'snli.config',
  'embedding_size': 300,
  'hidden_length': 64,
  'max_features': 50000,
  'max_hypothesis_length': 30,
  'max_premise_length': 30,
  'num_epochs': 5,
  'snli_link': 'https://nlp.stanford.edu/projects/snli/snli_1.0.zip',
  'snli_testfilename': 'snli_1.0_test.txt',
  'snli_trainfilename': 'snli_1.0_train.txt',
  'snli_validatefilename': 'snli_1.0_dev.txt',
  'snli_zipfilename': 'snli_1.0.zip',
  'word_embeddings_link': 'http://nlp.stanford.edu/data/glove.6B.zip',
  'word_embeddings_txtfilename': 'glove.6B.50d.txt',
  'word_embeddings_zipfilename': 'glove.6B.zip'}


In [20]:
# Load dataset manually
def prepare_snli_corpus():
    snli_link = FLAGS['snli_link']
    snli_zip_file = FLAGS['snli_zipfilename']
    snli_train_file = FLAGS['snli_trainfilename']
    snli_validate_file = FLAGS['snli_validatefilename']
    snli_test_file = FLAGS['snli_testfilename']
    
    if (not os.path.isfile(snli_zip_file)):
        print("Snli corpus not found. Downloading from site...")
        import urllib.request
        # download glove zip file
        urllib.request.urlretrieve(snli_link, snli_zip_file)
    print("Snli corpus file already downloaded. Extracting...")
    # extract train, validate and test files
    if (not os.path.isfile(snli_train_file)):
        unzip_single_file(snli_zip_file, snli_train_file)
        print("Extracted {}\n".format(snli_validate_file))
    else:
        print("{} already extracted.\n".format(snli_train_file))
    if (not os.path.isfile(snli_validate_file)):
        unzip_single_file(snli_zip_file, snli_validate_file)
        print("Extracted {}\n".format(snli_validate_file))
    else:
        print("{} already extracted.\n".format(snli_validate_file))
    if (not os.path.isfile(snli_test_file)):
        unzip_single_file(snli_zip_file, snli_test_file)
        print("Extracted {}\n".format(snli_test_file))
    else:
        print("{} already extracted.\n".format(snli_validate_file))
    return

def prepare_glove_embeddings():
    glove_link = FLAGS['word_embeddings_link']
    glove_zip_file = FLAGS['word_embeddings_zipfilename']
    glove_text_file = FLAGS['word_embeddings_txtfilename']
    
    if (not os.path.isfile(glove_zip_file) and not os.path.isfile(glove_text_file)):
        print("Glove embeddings not found. Downloading from site...")
        import urllib.request
        # download glove zip file
        urllib.request.urlretrieve(glove_link, glove_zip_file)
        print("Glove embeddings file downloaded.")
        # extract zip to text file
        unzip_single_file(glove_zip_file, glove_text_file)
    return

def unzip_single_file(zip_file_name, output_file_name):
    """
    If the outfile exists, don't recreate, else create from zipfile
    """
    if not os.path.isfile(output_file_name):
        import zipfile
        print("Unzipping glove embeddings {}..".format(zip_file_name))
        with open(output_file_name, "wb") as out_file:
            with zipfile.ZipFile(zip_file_name) as zipped:
                for info in zipped.infolist():
                    if output_file_name in info.filename:
                        with zipped.open(info) as requested_file:
                            out_file.write(requested_file.read())
                            print("Glove embeddings unzipped to {}".format(output_file_name))
                            return
    return

prepare_snli_corpus()
prepare_glove_embeddings()

Snli corpus file downloaded. Extracting...
Extracted snli_1.0_train.txt

Extracted snli_1.0_dev.txt

Extracted snli_1.0_test.txt

Sample word "the" with features array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)
Glove wordmap populated, found 400000 vectors


In [ ]:
def load_snli_data(filename):
    if not os.path.isfile(filename):
        print("ERROR: FILE NOT FOUND. EXITING...")
    else:
        import csv
        with open(filename, "r", encoding='utf-8') as data:
            train = csv.DictReader(data, delimiter='\t')
            premise_embeds = []
            hypothesis_embeds = []
            labels = []
            scores = []
            print("Parsing snli data to arrays...")
            for row in train:
                print(row)
                break
                
def load_glove_embeddings():
    global glove_wordmap
    global glove_wordmap_size

    glove_text_file = FLAGS['word_embeddings_txtfilename']
    printOne = True    

    with open(glove_text_file, "r", encoding='utf-8') as glove:
        for line in glove:
            values = line.split()
            word = values[0]
            # tensorflow only accepts arrays, not python lists
            featuresMatrix = np.asarray(values[1:], dtype='float32')
            # print a sample word with feature matrix
            if printOne:
                printOne = False
                print("Sample word \"{}\" with features {}".format(word, pp.pformat(featuresMatrix)))
            glove_wordmap[word] = featuresMatrix
    glove_wordmap_size = len(glove_wordmap)
    print("Glove wordmap populated, found %s vectors\n" % glove_wordmap_size)
    
load_glove_embeddings()
load_snli_data()